In [89]:
import sqlite3
import pandas as pd

In [90]:
con = sqlite3.connect('country_club.db')

Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [91]:
pd.read_sql_query("select name from Facilities where membercost > 0;", con)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


Q2: How many facilities do not charge a fee to members?

In [92]:
pd.read_sql_query("""select count(facid)
                    from Facilities
                    where membercost = 0""", con)

,count(facid)
0,4


Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [93]:
pd.read_sql_query("""select facid, name, membercost, monthlymaintenance
                    from Facilities
                    where membercost < 0.2*monthlymaintenance""", con)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator. 

In [94]:
pd.read_sql_query("""select *
                    from Facilities
                    where facid in (1, 5)""", con)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


 Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.

In [95]:
pd.read_sql_query("""SELECT name,
                    monthlymaintenance,
                    CASE 
                    WHEN monthlymaintenance > 100 THEN "expensive"
                    ELSE "cheap" END AS cheap_or_expensive
                    FROM Facilities""", con)

,name,monthlymaintenance,cheap_or_expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [96]:
pd.read_sql_query("""select firstname, surname
                    from Members
                    where joindate = (select max(joindate) from Members)""", con)

,firstname,surname
0,Darren,Smith


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [97]:
pd.read_sql_query("""SELECT Members.memid, (Members.surname || ", " || Members.firstname) AS name, Facilities.name AS facility
                    FROM Bookings
                    LEFT JOIN Members
                    ON Bookings.memid = Members.memid
                    INNER JOIN Facilities
                    ON Bookings.facid = Facilities.facid
                    WHERE Facilities.facid IN (0,1)
                    GROUP BY Members.memid
                    ORDER BY Members.surname;""", con)

,memid,name,facility
0,15,"Bader, Florence",Tennis Court 2
1,12,"Baker, Anne",Tennis Court 2
2,16,"Baker, Timothy",Tennis Court 1
3,8,"Boothe, Tim",Tennis Court 2
4,5,"Butters, Gerald",Tennis Court 1
5,22,"Coplin, Joan",Tennis Court 1
6,36,"Crumpet, Erica",Tennis Court 1
7,7,"Dare, Nancy",Tennis Court 1
8,13,"Farrell, Jemima",Tennis Court 1
9,28,"Farrell, David",Tennis Court 1


Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [98]:
pd.read_sql_query("""SELECT Facilities.name as facilityname, 
                    (Members.surname || ", " || Members.firstname) as membername,
                    (CASE WHEN Members.memid = 0 THEN Facilities.guestcost*Bookings.slots 
                    ELSE Facilities.membercost*Bookings.slots END) AS cost
                    FROM Bookings
                    LEFT JOIN Facilities
                    ON Bookings.facid = Facilities.facid
                    LEFT JOIN Members
                    ON Bookings.memid = Members.memid
                    WHERE Date(starttime) = Date("2012-09-14")
                    AND cost > 30
                    ORDER BY cost DESC;""", con)

,facilityname,membername,cost
0,Massage Room 2,"GUEST, GUEST",320.0
1,Massage Room 1,"GUEST, GUEST",160.0
2,Massage Room 1,"GUEST, GUEST",160.0
3,Massage Room 1,"GUEST, GUEST",160.0
4,Tennis Court 2,"GUEST, GUEST",150.0
5,Tennis Court 1,"GUEST, GUEST",75.0
6,Tennis Court 1,"GUEST, GUEST",75.0
7,Tennis Court 2,"GUEST, GUEST",75.0
8,Squash Court,"GUEST, GUEST",70.0
9,Massage Room 1,"Farrell, Jemima",39.6


Q9: This time, produce the same result as in Q8, but using a subquery.

In [99]:
pd.read_sql_query("""SELECT *
                    FROM (SELECT Facilities.name as facilityname, 
                    (Members.surname || ", " || Members.firstname) as membername,
                    (CASE WHEN Members.memid = 0 THEN Facilities.guestcost*Bookings.slots 
                    ELSE Facilities.membercost*Bookings.slots END) AS cost
                    FROM Bookings
                    LEFT JOIN Facilities
                    ON Bookings.facid = Facilities.facid
                    LEFT JOIN Members
                    ON Bookings.memid = Members.memid
                    WHERE Date(starttime) = Date("2012-09-14")
                    ORDER BY cost DESC)
                    WHERE cost > 30;""", con)

,facilityname,membername,cost
0,Massage Room 2,"GUEST, GUEST",320.0
1,Massage Room 1,"GUEST, GUEST",160.0
2,Massage Room 1,"GUEST, GUEST",160.0
3,Massage Room 1,"GUEST, GUEST",160.0
4,Tennis Court 2,"GUEST, GUEST",150.0
5,Tennis Court 1,"GUEST, GUEST",75.0
6,Tennis Court 1,"GUEST, GUEST",75.0
7,Tennis Court 2,"GUEST, GUEST",75.0
8,Squash Court,"GUEST, GUEST",70.0
9,Massage Room 1,"Farrell, Jemima",39.6


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [100]:
pd.read_sql_query("""SELECT Facilities.name as facilityname,
                    SUM(CASE WHEN memid = 0 THEN Bookings.slots*Facilities.guestcost
                    ELSE Bookings.slots*Facilities.membercost END) AS revenue 
                    FROM Bookings
                    LEFT JOIN Facilities
                    ON Bookings.facid = Facilities.facid
                    GROUP BY Facilities.facid
                    HAVING revenue < 1000
                    ORDER BY revenue;""", con)

,facilityname,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [101]:
con.close()